In [ ]:
import tensorflow as tf
import tensorflow_io as tfio
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:

audio_dir = Path("/home/guilhem/cours/birdsong-app/raw_dataset")
spectro_dir = Path("/home/guilhem/cours/birdsong-app/dataset")

spectro_dir.mkdir(parents=True, exist_ok=True)

def process_audio_file(audio_path):
    try:
        audio = tfio.audio.AudioIOTensor(str(audio_path))
        audio_tensor = tf.squeeze(audio[100:], axis=[-1])

        audio_tensor = tf.cast(audio_tensor, tf.float32) / 32768.0

        spectrogram = tfio.audio.spectrogram(
            audio_tensor, nfft=512, window=512, stride=256
        )

        plt.figure(figsize=(10, 4))
        plt.imshow(
            tf.math.log(spectrogram + 1e-6).numpy().T,
            aspect='auto',
            origin='lower',
            cmap='magma'
        )
        plt.axis('off')

        relative_path = audio_path.relative_to(audio_dir)
        output_path = spectro_dir / relative_path.with_suffix('.png')
        output_path.parent.mkdir(parents=True, exist_ok=True)

        plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close()
        print(f"✅ Sauvé : {output_path}")
    except Exception as e:
        print(f"❌ Échec pour {audio_path} : {e}")

all_audio_files = list(audio_dir.rglob("*.ogg"))

print(f"🔍 {len(all_audio_files)} fichiers trouvés.")

for audio_path in all_audio_files:
    process_audio_file(audio_path)

🔍 145 fichiers trouvés.


2025-05-02 14:43:08.381273: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2025-05-02 14:43:08.451931: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2025-05-02 14:43:08.898266: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-02 14:43:08.898699: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC748857.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC235919.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC375878.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC205521.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC132731.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC294900.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC622853.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC375876.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC205894.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC700635.png
✅ Sauvé : /home/guilhem/cours/birdsong-app/dataset/bird_spectrogram/bagwea1/XC364582.png
✅ Sauvé : /home/guilh

In [ ]:

dataset_dir = '/home/guilhem/cours/birdsong-app/dataset/bird_spectrogram'

training_set = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    image_size=(775, 308),  
    batch_size=32,
    label_mode='int',
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    image_size=(775, 308),  
    batch_size=32,
    label_mode='int',
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
)

Found 145 files belonging to 6 classes.
Using 116 files for training.
Found 145 files belonging to 6 classes.
Using 29 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))> <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [24]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = training_set.prefetch(buffer_size=AUTOTUNE)
val_ds = validation_set.prefetch(buffer_size=AUTOTUNE)

In [25]:


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation=None, input_shape=(775, 308, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.GlobalAveragePooling2D(),
    
    tf.keras.layers.Dense(128, activation=None),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(6, activation='softmax')
])


/home/guilhem/cours/birdsong-app/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=30
)

Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.2492 - loss: 2.0144 - val_accuracy: 0.2414 - val_loss: 2.5738
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.3881 - loss: 1.5008 - val_accuracy: 0.2069 - val_loss: 2.7269
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.5946 - loss: 1.2620 - val_accuracy: 0.2069 - val_loss: 2.9647
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.4664 - loss: 1.2807 - val_accuracy: 0.2069 - val_loss: 3.0870
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.5436 - loss: 1.2477 - val_accuracy: 0.2414 - val_loss: 3.3473
Epoch 6/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.5279 - loss: 1.2497 - val_accuracy: 0.2069 - val_loss: 3.3721
Epoch 7/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.4833 - loss: 1.2718 - val_accuracy: 0.0690 - val_loss: 2.9980
Epoch 8/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.5971 - loss: 1.1211 - val_accuracy: 0.1379 - val_loss: 2.6827
Epoch 9/30
4/4 